# West Ham v. Brighton - 2024-12-21 match analysis
*This notebook intends to walk through the theory and calculations I am currently using in my model to calculate probability of 1 x 2 result and a table of over / under goal projections. I will break the calculations out into utility functions stored in `../src/utils` for ease of use in future explorations. This is to document the calculations.*

### Instructions:

1. Ensure all cells in all notebooks in `/notebooks` directory have been run.
2. Define `home_side` and `away_side` in first cell. If making a comparison with odds from `03_retrieve_odds.ipynb` also enter matchday.
3. Click 'run all'
4. Probability for 1 x 2 and over / under can be viewed in the last cell
5. *Suggestion:* copy and paste notebook to `../output/< matchday_directory >/` and rename file. IMPORTANT: Update file path in lines 14 and 15 if needed if you make a copy.



### Import League Data & Define Sides

Define 'home_side' and 'away_side'
Ensure correct output



In [12]:
import pandas as pd
from scipy.stats import poisson
import numpy as np

"""
BUG: if copying from `notebooks/03_analysis_template_notebook.ipynb` to a directory in output
for storage, ensure the file path for reading the csv will need to be updated from '../data/processed/home_table.csv'
to '../../data/processed/home_table.csv' if frames are run again.

TLDR
if you are in ./notebooks/03_analysis_tempalte.csv you may run this cell as is. If you are not, check that following
read_csv commands read from the correct directory.
"""
home_dataframe = pd.read_csv('../data/processed/home_table.csv')
away_dataframe = pd.read_csv('../data/processed/away_table.csv')

"""
!!! INPUT TEAMS & MATCHDAY HERE !!!

wrapped in quotes, eg.

home_side = "Bournemouth"
away_side = "West Ham"
matchday = "2024-12-16" # (optional)

If there is a ValueError,
(1) check spelling against the error message. Must be exact.
(2) check capitalization
(3) check that it is wrapped in quotes

!!! INPUT TEAMS & MATCHDAY HERE !!!
"""
home_side = "West Ham"
away_side = "Brighton"
matchday = "2024-12-21" # (optional)

# validate input
clubs = set(home_dataframe['Team'].values)
if home_side not in clubs:
    raise ValueError(f"Invalid entry for 'home_side'. Expected one of {clubs}, but received '{home_side}'")
elif away_side not in clubs:
    raise ValueError(f"Invalid entry for 'away_side'. Expected one of {clubs}, but received '{away_side}'")
else:
    print(f"Ready to calculate probability for {home_side} v. {away_side}")

Ready to calculate probability for West Ham v. Brighton


### Calculate Projected Goals

In [13]:
# 2. grab their gpm_scored and gpm_conceded
home_side_gpm_scored = home_dataframe.loc[
    home_dataframe["Team"] == home_side,
    "gpm_scored"
].values[0]

home_side_gpm_conceded = home_dataframe.loc[
    home_dataframe["Team"] == home_side,
    "gpm_conceded"
].values[0]

print(home_side_gpm_scored, home_side_gpm_conceded)

away_side_gpm_scored = away_dataframe.loc[
    away_dataframe["Team"] == away_side,
    "gpm_scored"
].values[0]

away_side_gpm_conceded = away_dataframe.loc[
    away_dataframe["Team"] == away_side,
    "gpm_conceded"
].values[0]

print(away_side_gpm_scored, away_side_gpm_conceded)

# 3.
epl_home_avg_gpm_scored = round(sum(home_dataframe["gpm_scored"].values) / 20, 2)
epl_home_avg_gpm_conceded = round(sum(home_dataframe["gpm_conceded"].values) / 20, 2)
print(epl_home_avg_gpm_scored, epl_home_avg_gpm_conceded)

# 4. create a ratio (attack and defense rating) of their gpm to the league average gpm
home_attack_rating = home_side_gpm_scored / epl_home_avg_gpm_scored
away_defense_rating = away_side_gpm_conceded / epl_home_avg_gpm_scored
away_attack_rating = away_side_gpm_scored / epl_home_avg_gpm_conceded
home_defense_rating = home_side_gpm_conceded / epl_home_avg_gpm_conceded

print(home_attack_rating, away_defense_rating, away_attack_rating, home_defense_rating)

# home projected goals
home_projected_goals = home_attack_rating * away_defense_rating * epl_home_avg_gpm_scored
# away projected goals
away_projected_goals = away_attack_rating * home_defense_rating * epl_home_avg_gpm_conceded
print("\n ================== projected_goals ==================== \n")
print(f"HOME {home_side} projected goals:", home_projected_goals)
print(f"AWAY {away_side} projected goals: ", away_projected_goals)

1.5 2.0
1.625 1.625
1.61 1.4
0.9316770186335404 1.0093167701863353 1.1607142857142858 1.4285714285714286

 ================== projected_goals ==================== 

HOME West Ham projected goals: 1.5139751552795029
AWAY Brighton projected goals:  2.3214285714285716


### Poisson Distribution of Goal Probabilities

In [14]:
# Poisson probability for home team goals 0 - 8
home_score_prob = []
away_score_prob = []
for i in range(9):
    prob = poisson.pmf(i, home_projected_goals)
    home_score_prob.append(prob)
for i in range(9):
    prob = poisson.pmf(i, away_projected_goals)
    away_score_prob.append(prob)

home_score_prob = np.array(home_score_prob)
away_score_prob = np.array(away_score_prob)
# print(home_score_prob)
# print(away_score_prob)

exact_score_prob = np.outer(away_score_prob, home_score_prob)
print(f"{home_side} v. {away_side}", exact_score_prob)

West Ham v. Brighton [[2.15926192e-02 3.26906890e-02 2.47464455e-02 1.24885012e-02
  4.72682014e-03 1.43125765e-03 3.61148087e-04 7.81098902e-05
  1.47820542e-05]
 [5.01257231e-02 7.58890995e-02 5.74471056e-02 2.89911635e-02
  1.09729753e-02 3.32256240e-03 8.38379489e-04 1.81326531e-04
  3.43154829e-05]
 [5.81816429e-02 8.80855619e-02 6.66796761e-02 3.36504577e-02
  1.27364892e-02 3.85654565e-03 9.73119049e-04 2.10468295e-04
  3.98304712e-05]
 [4.50215094e-02 6.81614467e-02 5.15973684e-02 2.60390446e-02
  9.85561665e-03 2.98423175e-03 7.53008788e-04 1.62862371e-04
  3.08211979e-05]
 [2.61285546e-02 3.95579824e-02 2.99449013e-02 1.51119455e-02
  5.71977752e-03 1.73192021e-03 4.37014029e-04 9.45183403e-05
  1.78873024e-05]
 [1.21311146e-02 1.83662061e-02 1.39029899e-02 7.01626043e-03
  2.65561099e-03 8.04105813e-04 2.02899371e-04 4.38835151e-05
  8.30481896e-06]
 [4.69358601e-03 7.10597261e-03 5.37913299e-03 2.71462457e-03
  1.02746854e-03 3.11112368e-04 7.85027326e-05 1.69787410e-05
  3

### Under Market

In [15]:
thresholds = [0.5, 1.5, 2.5, 3.5]
sums = []

for t in thresholds:
    mask = np.add.outer(
        np.arange(
            exact_score_prob.shape[0]
        ),
        np.arange(
            exact_score_prob.shape[1]
        )
    ) <= t
    # print(mask)
    sums.append(exact_score_prob[mask].sum())

# print(sums)
under_market_df = pd.DataFrame({
    "Goals": thresholds,
    "Prob": sums
})
under_market_df["Implied Odds"] = 1 / under_market_df["Prob"]
print("~  ~ under market ~  ~\n", f"{home_side} v. {away_side}\n", under_market_df)

~  ~ under market ~  ~
 West Ham v. Brighton
    Goals      Prob  Implied Odds
0    0.5  0.021593     46.312121
1    1.5  0.104409      9.577716
2    2.5  0.263226      3.799014
3    3.5  0.466269      2.144685


In [16]:
thresholds = [0.5, 1.5, 2.5, 3.5]
sums = []

for t in thresholds:
    mask = np.add.outer(
        np.arange(
            exact_score_prob.shape[0]
        ),
        np.arange(
            exact_score_prob.shape[1]
        )
    ) >= t
    sums.append(exact_score_prob[mask].sum())

# print(sums)
over_market_df = pd.DataFrame({
    "Goals": thresholds,
    "Prob": sums
})

over_market_df["Implied Odds"] = 1 / over_market_df["Prob"]
print("~  ~ over market ~  ~\n", f"{home_side} v. {away_side}\n", over_market_df)

~  ~ over market ~  ~
 West Ham v. Brighton
    Goals      Prob  Implied Odds
0    0.5  0.977693      1.022816
1    1.5  0.894877      1.117472
2    2.5  0.736060      1.358586
3    3.5  0.533017      1.876113


### 1 x 2 Probability

In [17]:
# Poisson probability for home team goals 0 - 8
home_score_prob = []
away_score_prob = []
for i in range(9):
    prob = poisson.pmf(i, home_projected_goals)
    home_score_prob.append(prob)
for i in range(9):
    prob = poisson.pmf(i, away_projected_goals)
    away_score_prob.append(prob)

home_score_prob = np.array(home_score_prob)
away_score_prob = np.array(away_score_prob)
# print(home_score_prob)
# print(away_score_prob)

exact_score_prob = np.outer(away_score_prob, home_score_prob)

# print(exact_score_prob)
home_win_prob = np.sum(np.triu(exact_score_prob, k=1))
away_win_prob = np.sum(np.tril(exact_score_prob, k=-1))
draw_prob = np.trace(exact_score_prob)

# prepare data in readable dataframe
data = {
    "winner": [f"{home_side} (1)", "Draw (x)", f"{away_side} (2)"],
    "prob": [home_win_prob, draw_prob, away_win_prob],
}
result_df = pd.DataFrame(data)
result_df['implied_odds'] = 1 / result_df['prob']
print("      ~  ~ moneyline market ~  ~\n", result_df)

      ~  ~ moneyline market ~  ~
          winner      prob  implied_odds
0  West Ham (1)  0.246137      4.062783
1      Draw (x)  0.196809      5.081075
2  Brighton (2)  0.556340      1.797461


# Full Results Summary:

In [18]:
# helper function to translate discrepency between Understat and Odds-Api naming
def translate(string):

    translations = {
        "Brighton": "Brighton and Hove Albion",
        "Ipswich": "Ipswich Town",
        "Leicester": "Leicester City",
        "Tottenham": "Tottenham Hotspur",
        "West Ham": "West Ham United"
    }

    if string in translations:
        return translations[string]
    return string

home_side, away_side = translate(home_side), translate(away_side)

match_string = f"{home_side} v. {away_side}"

print(f"CALCULATED PROBABILITIES FOR {home_side} v. {away_side}")
print("\n\n        ~ ~ moneyline market ~ ~\n\n", result_df)
print("\n\n          ~ ~ over market ~ ~ \n\n", over_market_df)
print("\n\n          ~ ~ under market ~ ~ \n\n", under_market_df)

try:
    odds_df = pd.read_csv(f"../data/processed/odds/odds_matchday_{matchday}.csv")
    filtered_odds = odds_df.loc[odds_df["match"] == match_string]
    implied = {
        "1": 1 / filtered_odds["1"],
        "x": 1 / filtered_odds["x"],
        "2": 1 / filtered_odds["2"],
        "point": filtered_odds["ovr_und_point"],
        "over": 1 / filtered_odds["over"],
        "under": 1 / filtered_odds["under"]
    }
    implied_odds_df = pd.DataFrame(implied)

    print(f"\n ~~ MyBookie Odds for {match_string} ~~\n",filtered_odds.drop(columns=["match"]))
    print(f"\n ~~ MyBookie Odds IMPLIED for {match_string} ~~\n", implied_odds_df)
except:
    print(f"No data for sportsbook odds on matchday {matchday}. To view against real odds, run `notebooks/03_retrieve_odds.ipynb` ran correctly and ensure matchday is correct in the first cell of this notebook")


CALCULATED PROBABILITIES FOR West Ham United v. Brighton and Hove Albion


        ~ ~ moneyline market ~ ~

          winner      prob  implied_odds
0  West Ham (1)  0.246137      4.062783
1      Draw (x)  0.196809      5.081075
2  Brighton (2)  0.556340      1.797461


          ~ ~ over market ~ ~ 

    Goals      Prob  Implied Odds
0    0.5  0.977693      1.022816
1    1.5  0.894877      1.117472
2    2.5  0.736060      1.358586
3    3.5  0.533017      1.876113


          ~ ~ under market ~ ~ 

    Goals      Prob  Implied Odds
0    0.5  0.021593     46.312121
1    1.5  0.104409      9.577716
2    2.5  0.263226      3.799014
3    3.5  0.466269      2.144685

 ~~ MyBookie Odds for West Ham United v. Brighton and Hove Albion ~~
       1    x     2  ovr_und_point  over  under
2  2.82  3.6  2.29            3.5  2.19   1.63

 ~~ MyBookie Odds IMPLIED for West Ham United v. Brighton and Hove Albion ~~
          1         x         2  point      over     under
2  0.35461  0.277778  0.436

# Conclusion and Pick
GUTS: Brighton to win on the road. West Ham is a club that I do not like wagering against, but I like the line. These are the picks I'm looking for with this model.
### My Pick:

**Brighton @ 2.29** -- *1 unit* 